In [ ]:
import warnings
warnings.filterwarnings('ignore')
from helpers import *
import numpy as np
import plotly.express as px

# Estimating Pandemic Risk

Here, I estimate the expected number of deaths this century from the following types of pandemics:
1. Natural pandemics (based on estimates from [Marani et al. 2021](https://doi.org/10.1073/pnas.2105482118))
2. Accidental pandemics (based on estimates from [Klotz 2021](https://armscontrolcenter.org/wp-content/uploads/2017/04/LWC-paper-final-version-for-CACNP-website.pdf))
3. Deliberate pandemics (based on estimates from [Esvelt 2022](https://dam.gcsp.ch/files/doc/gcsp-geneva-paper-29-22?_gl=1*ieur8b*_ga*MTk1NzA0MTU3My4xNjk2NzcyODA0*_ga_Z66DSTVXTJ*MTY5Njc3MjgwNC4xLjAuMTY5Njc3MjgwNi41OC4wLjA.))

In [ ]:
# Define vars
params = Params()

I will estimate the expected number of deaths this century i.e. from 2023 to 2100. For simplicity, I will assume a fixed population of 9.2 billion people over this time period (based on [UN population projections](https://www.worldometers.info/world-population/world-population-projections/)):

$
\text{Average population} = \frac{\text{Starting population} + \text{Population in 2100}}{2} = \frac{8.0 \text{ billion} + 10.3 \text{ billion}}{2} = 9.2 \text{ billion}
$

For the estimates, I will be using Monte Carlo simulations. These parameters which are used for multiple of the estimates are shown below:

In [ ]:
params.print_category('Global')

## 1. Estimating Natural Pandemic Risk
[Marani et al. 2021](https://doi.org/10.1073/pnas.2105482118) assembled a dataset of epidemics that have occured since 1500. The dataset includes the start and end dates of each epidemic and the number of deaths. The dataset only contains epidemics were not currently active (e.g., AIDS/HIV, malaria, and COVID-19 were excluded), and epidemics that were ended by the introduction of vaccines or effective treatments were excluded.

The dataset is visualised below:

In [ ]:
marani_xls = params.Natural.dataset.val
marani_df, disease_totals = load_and_preprocess_natural_data(marani_xls)
color_map = generate_color_map(disease_totals)
fig = plot_disease_timeline(marani_df, disease_totals, color_map)

Marani et al. showed that the when the epidemic intesity, that is the number of deaths per year, is plotted against the exceedance probability, which is the probability that the epidemic intensity will be exceeded in a given year, the data is well described by a generalized Pareto distribution (GPD).

In [ ]:
fig = plot_intensity_exceedance_probability(marani_df)

## 2. Estimating Accidental Pandemic Risk

We'll be examining the potential risk of a pandemic from the accidental release of viruses from research facilities. This analysis will involve three main sections:

1. Probability a single facility in a single year seeds a pandemic
2. Expected number of accidental pandemics this century
3. Expected number of deaths from accidental pandemics this century

Note, limitations of this analysis include:
- The results are highly dependent on the probabilities of release computed in [Klotz 2020](https://armscontrolcenter.org/wp-content/uploads/2020/03/Quantifying-the-risk-9-17-Supplementary-material-at-end.pdf)
- We are only considering the risk from highly pathogenic avian influenza (HPAI) viruses, and not other types of viruses
- We are only considering the risk from research facilities, and not other sources of accidental release
- We are assuming that the number of facilities conducting HPAI research will remain constant over the this century


In [ ]:
params.print_category('Accidental')

### 2.1 Probability a single facility in a single year seeds a pandemic

The probability that a single facility seeds a pandemic in a single year is given by:

$
P_{\text{single_pandemic}} = P_{\text{release}} \times P_{\text{seeds_pandemic}}
$

Where:
- $P_{\text{release}}$ is the probability of community release from a single facility in a single year.
- $P_{\text{seeds_pandemic}}$ is the probability that a virus release seeds a pandemic. Since $P_{\text{seeds_pandemic}}$ is given as a range, we will sample from a uniform distribution between 0.05 and 0.4.

In [ ]:
P_seeds_pandemic_min = params.Accidental.P_seeds_pandemic_min.val
P_seeds_pandemic_max = params.Accidental.P_seeds_pandemic_max.val
num_simulations = params.Global.num_simulations.val
P_release = params.Accidental.P_release.val
accidental_colour = params.Accidental.colour.val

P_seeds_pandemic = np.random.uniform(P_seeds_pandemic_min, P_seeds_pandemic_max, num_simulations)
P_single_pandemic = P_release * P_seeds_pandemic

# Plot the distribution of P_single_pandemic
fig = plot_P_single_pandemic_hist(P_single_pandemic, accidental_colour)

### 2.2. Expected number of accidental pandemics this century

The expected number of pandemics seeded by any facility in a $num\_years$ is given by:

$
E[\text{#Pandemics}] = P_{\text{single_pandemic}} \times \text{num_years} \times \text{num_facilities}
$

Where:
- $\text{num_years}$ is the number of years in the simulation (typically set to 100 for a century).
- $\text{num_facilities}$ is the number of facilities in the world.
- $P_{\text{single_pandemic}}$ is the probability that a single facility seeds a pandemic in a single year.

In [ ]:
num_years = params.Global.num_years.val
num_facilities = params.Accidental.num_facilities.val

E_accidental_pandemics = P_single_pandemic * num_years * num_facilities

# Plot the distribution of E_accidental_pandemics
fig = plot_E_accidental_pandemics_hist(E_accidental_pandemics, accidental_colour)

### 2.3. Expected number of deaths from accidental pandemics this century

For each pandemic, the number of deaths is calculated as:

$
E[\text{#Deaths} ]= E[\text{#Pandemics}] \times \text{Population} \times \text{Infection Rate} \times \text{CFR}
$

Where:
- $\text{Population}$ is the world population at the time of the pandemic.
- $\text{Infection Rate}$ is the infection rate of the pandemic.
- $\text{CFR}$ is the case fatality rate of the pandemic.

In [ ]:
population = params.Global.population.val
infection_rate = params.Accidental.infection_rate.val
fatality_rate = params.Accidental.fatality_rate.val

E_accidental_deaths = E_accidental_pandemics * population * infection_rate * fatality_rate

# Plot the distribution of E_accidental_deaths
fig = plot_E_accidental_deaths_hist(E_accidental_deaths, accidental_colour)

## 3. Estimating Deliberate Pandemic Risk

In [ ]:
params.print_category('Deliberate')

In [ ]:
gtd_xls = params.Deliberate.dataset.val
deaths_per_attack = params.Deliberate.deaths_per_attack.val
population_us_1995 = params.Deliberate.population_us_1995.val
num_indv_capability = params.Deliberate.num_indv_capability.val
deliberate_multiplier_max = params.Deliberate.deliberate_multiplier_max.val
deliberate_colour = params.Deliberate.colour.val

# Load and preprocess data
gtd_df = load_and_preprocess_deliberate_data(gtd_xls, deaths_per_attack)
num_events = len(gtd_df)
frac_invd_intent = num_events / population_us_1995

# Plot 
fig = plot_deaths_per_attack_scatter(gtd_df, deaths_per_attack, num_events, frac_invd_intent)

In [ ]:
total_individuals = calculate_individual_capability(params)
fig = plot_capability_growth(params, deliberate_colour)

In [ ]:
# Expected number of individuals with the intent to cause mass harm this century
num_indv_capability = total_individuals[-1]
E_num_indv_capability_per_century = num_indv_capability * frac_invd_intent
deliberate_multiplier = np.random.uniform(1, deliberate_multiplier_max, num_simulations)
E_deliberate_deaths = E_num_indv_capability_per_century * population * infection_rate * fatality_rate * deliberate_multiplier

# Plot the distribution of E_deliberate_deaths
fig = plot_E_deliberate_deaths_hist(E_deliberate_deaths, deliberate_colour)